# VacationPy
----




------------------------------------
- This code took about 18 seconds to run
- i've ran it a couple times and there seems to be a bias around africa, the middle east, and india. It could be caused by the random coordinates generated in weather.py or the fact that these regions are huge, or it could be the weather based filters I applied

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

from full_country import full_name_dict

### this allows me to keep my keys in a folder outside my repo
import sys
try:
    sys.path.insert(1,'/Users/Trav/Desktop/Coding/KEY_PACKAGE')
    from api_keys import g_key
except:
    from api_keys import g_key





### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
## import csv to dataframe, add try block to deal with directory issues
csv_import_path = os.path.join('..','Weather','Export Folder','city_data.csv')
city_weather_df=pd.read_csv(csv_import_path)

city_weather_df.head()

# # ### using this to limit the request, remove once completed
# city_weather_df=city_weather_df.head()
# city_weather_df.head(20)

,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp
0,-31.36,60.51,saint-philippe,re,"(-21.35, 55.7666667)",-21.350000,55.766667,70.97,50,75,21.92,75.20
1,-74.85,19.80,bredasdorp,za,"(-34.532154, 20.040308)",-34.532154,20.040308,55.40,71,34,6.58,55.40
2,-16.77,-39.21,porto seguro,br,"(-16.433333, -39.083333)",-16.433333,-39.083333,75.20,83,0,6.93,75.20
3,-54.59,12.64,hermanus,za,"(-34.418696, 19.234464)",-34.418696,19.234464,59.00,70,0,4.00,59.00
4,-89.98,-161.79,mataura,pf,"(-23.3666667, -149.4666667)",-23.366667,-149.466667,67.78,91,100,21.03,67.78


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps with key
gmaps.configure(api_key=g_key)

In [4]:

#defines coordinates to plot and weight is humididty

locations=city_weather_df[['Latitude','Longitude']]
weighty=city_weather_df['Humidity']

fig=gmaps.figure(center=(0,0),zoom_level=2)
heat=gmaps.heatmap_layer(locations,weights=weighty,dissipating=True,max_intensity=200,point_radius=20)
fig.add_layer(heat)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#narrows down the dataframe to a dataframe with ideal weather
ideal_weather = city_weather_df.loc[(city_weather_df['Max Temp']>70)&(city_weather_df['Max Temp']<80)&(city_weather_df['Wind Speed']<10)&(city_weather_df['Cloudiness']==0),:]

#drops bad data
ideal_weather.dropna()


ideal_weather.head()

,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp
2,-16.77,-39.21,porto seguro,br,"(-16.433333, -39.083333)",-16.433333,-39.083333,75.20,83,0,6.93,75.20
40,-61.74,-135.72,rikitea,pf,"(-23.1202778, -134.9691667)",-23.120278,-134.969167,75.83,70,0,5.21,75.83
89,-15.54,62.92,grand gaube,mu,"(-20.0063889, 57.6608333)",-20.006389,57.660833,73.99,42,0,8.99,73.99
144,22.06,7.72,arlit,ne,"(18.7325, 7.3680556)",18.732500,7.368056,71.85,22,0,8.72,71.85
163,9.93,18.15,sarh,td,"(9.15, 18.3833333)",9.150000,18.383333,73.40,94,0,4.05,73.40


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#simplifies and defines the hotel_df
hotel_df=ideal_weather.drop(['Approximate Lat','Approximate Lon','Exact Coordinates'],axis=1).reset_index(drop=True)

#creates blank series that will be filled in
hotel_df['Hotel Name']=""
hotel_df['Hotel Lat']=""
hotel_df['Hotel Lon']=""
# #renaming and making uppercase
# hotel_df=hotel_df.rename(columns={'City Name':'City','Country Code':'Country'})
hotel_df['Country Code']=hotel_df['Country Code'].str.upper()

hotel_df=hotel_df.rename(columns={'City Name':'City'})
#creating a dataframe from the countries full names
country_name_df=pd.DataFrame.from_dict(full_name_dict,orient='index').reset_index()
country_name_df=country_name_df.rename(columns={'index':'Country Code',0:'Country'})

#merging in the country names

hotel_df=pd.merge(country_name_df,hotel_df,on="Country Code",how='right')

hotel_df.head()

,Country Code,Country,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp,Hotel Name,Hotel Lat,Hotel Lon
0,BR,Brazil,porto seguro,-16.433333,-39.083333,75.20,83,0,6.93,75.20,,,
1,BR,Brazil,belmonte,-15.850000,-38.900000,75.20,83,0,6.93,75.20,,,
2,ET,Ethiopia,bedesa,8.900000,40.783333,71.19,55,0,1.77,71.19,,,
3,MG,Madagascar,tsihombe,-25.300000,45.483333,78.73,41,0,9.35,78.73,,,
4,MU,Mauritius,grand gaube,-20.006389,57.660833,73.99,42,0,8.99,73.99,,,


In [7]:

### finds the closest hotel and fills in the hotel name and coordinates to the dataframe
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "Hotel"
target_radius = 5000
target_type = "Hotel"

for i,r in hotel_df.iterrows():
    c_lat=hotel_df['Latitude'][i]
    c_lon = hotel_df['Longitude'][i]
    target_coordinates =f'{c_lat}, {c_lon}' 
    
    param = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    places_json=requests.get(base_url,params=param).json()
    ## the city tessalit is literally in the middle of the sahara desert and returns no results, so this is a work around.

    if places_json['status']=='ZERO_RESULTS':
        hotel_df['Hotel Name'][i]=np.nan
    else:
        hotel_df['Hotel Name'][i]=places_json["results"][0]['name']
        hotel_df['Hotel Lat'][i]=places_json["results"][0]['geometry']['location']['lat']
        hotel_df['Hotel Lon'][i]=places_json["results"][0]['geometry']['location']['lng']


## Dropping the cities that returned no results 
hotel_df=hotel_df.dropna()

hotel_df.head(30)


,Country Code,Country,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp,Hotel Name,Hotel Lat,Hotel Lon
0,BR,Brazil,porto seguro,-16.433333,-39.083333,75.20,83,0,6.93,75.20,Portal Beach Hotel,-16.4079,-39.0485
1,BR,Brazil,belmonte,-15.850000,-38.900000,75.20,83,0,6.93,75.20,Pousada do Diogo,-15.8633,-38.8755
3,MG,Madagascar,tsihombe,-25.300000,45.483333,78.73,41,0,9.35,78.73,Restaurant Le Paradis du sud,-25.3187,45.4837
4,MU,Mauritius,grand gaube,-20.006389,57.660833,73.99,42,0,8.99,73.99,LUX Grand Gaube Resort & Villas,-20.0023,57.6598
5,NE,Niger,arlit,18.732500,7.368056,71.85,22,0,8.72,71.85,Hôtel Telwa Bungalow,18.739,7.38945
6,NE,Niger,diffa,13.315556,12.608889,77.04,27,0,8.19,77.04,Complexe Hôtelier Du 18 Décembre,13.3127,12.5904
7,NG,Nigeria,bida,9.083333,6.016667,74.37,63,0,1.99,74.37,House 7 Resort,9.06385,5.9981
8,PF,French Polynesia,rikitea,-23.120278,-134.969167,75.83,70,0,5.21,75.83,Pension Bianca & Benoit,-23.1277,-134.966
9,TD,Chad,sarh,9.150000,18.383333,73.40,94,0,4.05,73.40,Hôtel des Chasses,9.14226,18.3899


In [8]:
# # #create a new folder and exports the dataframe

def dataframe_to_csv(df):
    current_directory=os.getcwd()
    new_folder_path =os.path.join(current_directory,r'Export Folder')
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
    csv_export_path = os.path.join(new_folder_path,'hotel_data.csv')
    df.to_csv(csv_export_path,index=False)
dataframe_to_csv(hotel_df)

In [9]:
## import the csv file you just saved
def import_path():
    current_directory=os.getcwd()
    import_path.new_folder_path =os.path.join(current_directory,r'Export Folder')
    csv_export_path = os.path.join(import_path.new_folder_path,'hotel_data.csv')
    return csv_export_path

im_path=import_path()
csv_imp_df=pd.read_csv(im_path,delimiter=',')
csv_imp_df.head()

,Country Code,Country,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp,Hotel Name,Hotel Lat,Hotel Lon
0,BR,Brazil,porto seguro,-16.433333,-39.083333,75.20,83,0,6.93,75.20,Portal Beach Hotel,-16.407943,-39.048535
1,BR,Brazil,belmonte,-15.850000,-38.900000,75.20,83,0,6.93,75.20,Pousada do Diogo,-15.863311,-38.875494
2,MG,Madagascar,tsihombe,-25.300000,45.483333,78.73,41,0,9.35,78.73,Restaurant Le Paradis du sud,-25.318744,45.483670
3,MU,Mauritius,grand gaube,-20.006389,57.660833,73.99,42,0,8.99,73.99,LUX Grand Gaube Resort & Villas,-20.002344,57.659766
4,NE,Niger,arlit,18.732500,7.368056,71.85,22,0,8.72,71.85,Hôtel Telwa Bungalow,18.738976,7.389448


In [10]:


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in csv_imp_df.iterrows()]

# hotel_info


In [11]:
### plotting heat map and hotel locations
y_locations=city_weather_df[['Latitude','Longitude']]
x_locations=csv_imp_df[['Hotel Lat','Hotel Lon']]
x_weighty=city_weather_df['Humidity']



x_fig=gmaps.figure(center=(0,0),zoom_level=2)
x_heat=gmaps.heatmap_layer(y_locations,weights=x_weighty,dissipating=True,max_intensity=200,point_radius=20)
x_marks=gmaps.marker_layer(x_locations,info_box_content=hotel_info)
x_fig.add_layer(x_marks)
x_fig.add_layer(x_heat)

x_fig

Figure(layout=FigureLayout(height='420px'))